# Pipeline Genotipagem do virus SarsCov-2
## Grupo:
- Julia Beltramini
- Kaira Cristina Peralis Tomaz
- Leonardo V. Azevedo
- Mélcar Collodetti
- Sara Reis
- Victor Aldair

## Preparar o ambiente

Utilize o comando abaixo para instalar os programas necessários para executar o pipeline de genotipagem viral.

In [ ]:
%%bash
sudo apt install tree fastqc bwa samtools bedtools freebayes
pip install cutadapt

Testa se o programa foi instalado corretamente (deve aparecer uma mensagem com o manual do programa)

In [ ]:
!samtools


## Download dos arquivos necessários da amostra (FASTQ)

Escolha de qual paciente você seguirá para diagnóstico:

In [ ]:
PACIENTE_R1 = "https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/TCC-metagenomica/patient_joao_VIROMA_S21_R1_001.fastq.gz"
PACIENTE_R2 = "https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/TCC-metagenomica/patient_joao_VIROMA_S21_R2_001.fastq.gz"

**ATENÇÃO**

Edite está célula para PACIENTE1 ou PACIENTE2 de acordo com qual paciente deseje prosseguir com a análise e diagnóstico.  
Em seguida, execute a célula abaixo e aguarde o download dos arquivos. Demora cerca de 3 minutos.

In [ ]:
%%bash
git clone https://github.com/circulosmeos/gdown.pl.git
./gdown.pl/gdown.pl https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/TCC-metagenomica/patient_joao_VIROMA_S21_R1_001.fastq.gz PACIENTE1_VIROMA_S21_R1_001.fastq.gz

In [ ]:
%%bash
git clone https://github.com/circulosmeos/gdown.pl.git
./gdown.pl/gdown.pl https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/TCC-metagenomica/patient_joao_VIROMA_S21_R2_001.fastq.gz PACIENTE1_VIROMA_S21_R2_001.fastq.gz

In [ ]:
%%bash
# Download SARS-COV-2 Reference
wget -nv https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/NC_045512.fasta

2024-02-13 22:27:43 URL:https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/NC_045512.fasta [30429/30429] -> "NC_045512.fasta" [1]


Organizar arquivos nas suas respectivas pastas





In [ ]:
%%bash
mkdir -p logs fastq passedQC reference mapped variants coverage
mv *fastq.gz fastq/
mv NC_045512.fasta reference/


In [ ]:
# Coletar nome da amostra em uma variável
temp = !basename fastq/*_R1* _R1_001.fastq.gz
SAMPLE = temp[0]
print(SAMPLE)

Verifique se os arquivos estão salvos corretamente no diretório de trabalho. O comando *tree* mostra as pastas e seus respectivos conteúdos na forma de uma árvore.

In [ ]:
!tree

## Controle de Qualidade das Sequências com FASTQC

O comando abaixo irá executar um programa chamado FASTQC, que é muito utilizado para gerar relatórios sobre a qualidade dos seus dados de sequenciamento.

In [ ]:

!fastqc fastq/{SAMPLE}_R1_001.fastq.gz fastq/{SAMPLE}_R2_001.fastq.gz

Novamente, verifique com o comando abaixo que dois arquivos .HTML foram criados dentro da pasta FASTQ. Você pode baixá-los através da janela a esquerda e abri-los no seu navegador de preferência.

In [ ]:
!tree

## Limpeza das sequências com cutadapt

Um dos software mais utilizados para limpeza das sequências de NGS se chama *cutadapt*. Vamos utilizá-los para trimar as pontas das sequências e para filtrar as sequências menores do que 50pb.  
Os parâmetros de limpeza podem variar de corrida a corrida ou mesmo do tipo de tecnologia de sequenciamento. Olhar os relatórios do FASTQC gerado acima pode te dar boas pistas de como fazer essa limpeza. Olhe lá e tente parâmetros diferentes aqui. Lembrando que se você digitar *fastqc -h* em uma das células, o prompt te retornará o manual de uso do software.

In [ ]:

!cutadapt -u 5 -U 5 -u -9 -U -9 -m 50 -o passedQC/{SAMPLE}_cleaned_R1.fastq -p passedQC/{SAMPLE}_cleaned_R2.fastq fastq/{SAMPLE}_R1_001.fastq.gz fastq/{SAMPLE}_R2_001.fastq.gz

## Mapeamento dos reads no genoma de referência do SARS-CoV-2

Para fazer o mapeamento dos reads de sequenciamento limpos, iremos utilizar um software chamado BWA. Ele é muito utilizado para fazer alinhamento de sequencias curtas contra um genoma de referência, que no nosso caso é o genoma do SARS-CoV-2.

Primeiramente, é necessário preparar a referência para o alinhamento. É o que chamamos de indexação da referência.

In [ ]:
%%bash
bwa index reference/NC_045512.fasta

Após a indexação, podemos usar o comando que faz o mapeamento chamado *bwa mem*. Lembrando sempre que o programa precisa dos fastqs limpos e da referência como entrada. Este comando pode demorar cerca de 2 minutos, seja paciente.

In [ ]:

!bwa mem reference/NC_045512.fasta passedQC/{SAMPLE}_cleaned_R1.fastq passedQC/{SAMPLE}_cleaned_R2.fastq > mapped/{SAMPLE}_mapped_sarscov2.sam


Perceba que ele gera como resultado na pasta *mapped* um arquivo no formato .SAM. O sam é a versão arquivo de texto do arquivo .BAM que iremos gerar logo mais.

## Organizar o SAM e gerar o BAM

O programa samtools é uma suite de ferramenta para lidar com os arquivos fasta e de mapeamento. Nós vamos utilizar o *samtools sort* para ordenar e gerar o arquivo .BAM.

In [ ]:

!samtools sort mapped/{SAMPLE}_mapped_sarscov2.sam -o mapped/{SAMPLE}_mapped_sarscov2_sorted.bam

## Gerar dados de cobertura para o mapeamento

A suite de comandos chamada *bedtools* tem uma série de ferramentas para lidar com dados de cobertura dos alinhamentos. Nós iremos utilizar os comandos abaixo para preparar um arquivo .BED com a maior região contínua do genoma do SARS-CoV-2 com cobertura de leituras de sequenciamento.

In [ ]:

!bedtools bamtobed -i mapped/{SAMPLE}_mapped_sarscov2_sorted.bam > coverage/mapped_sarcov2.bed
!bedtools merge -i coverage/mapped_sarcov2.bed >coverage/mapped_sarscov2_merged.bed
!bedtools sort -i coverage/mapped_sarscov2_merged.bed >coverage/mapped_sarscov2_merged_sorted.bed

Após gerar este .BED, vamos utilizar o comando *bedtools coverage* para gerar a cobertura média nesta região contínua.

In [ ]:

!bedtools coverage -a coverage/mapped_sarscov2_merged_sorted.bed \
-b mapped/{SAMPLE}_mapped_sarscov2_sorted.bam -mean \
>coverage/results_coverage.bed

O arquivo *results_coverage.bed* na pasta *coverage* possui linhas, mostrando separados por tab:  
`cromossomo  posição_inicio  pos_fim  cobertura_vertical_média`

In [ ]:
%%bash
cat coverage/results_coverage.bed

Veja novamente os arquivos novos criados na pasta *coverage*

In [ ]:
!tree

## Fazer a chamada de variantes com o Freebayes

Agora, tendo como base o arquivo .BAM de mapeamento, vamos fazer a chamada de variantes. Para tal, iremos utilizar o software *freebayes*. Tenha em mente que existem várias ferramentas para esta finalidade, e estamos utilizando aqui uma destas opções com parâmetros default. Tenha paciência, o comando demora cerca de 3 minutos.

In [ ]:

!freebayes -f reference/NC_045512.fasta -p 1 mapped/{SAMPLE}_mapped_sarscov2_sorted.bam > variants/{SAMPLE}_variants.vcf

Veja o arquivo .VCF gerado dentro da pasta *variants*.

In [ ]:
!tree

Listar as variantes no VCF, caso queiram ver o arquivo bruto gerado.

In [ ]:
!cat variants/{SAMPLE}_variants.vcf

## Anotação do VCF com base no genoma de referência de SARS-CoV-2

O arquivo VCF apresenta as coordenadas de uma mutação ou variante, mas não apresenta outras informações interessantes como por exemplo: Qual o tipo de variante (SNP, indel, etc), é sinônima?, está em qual gene?, e qual a alteração na proteína. Para fazer este tipo de anotação que é específica de cada genoma de referência, vamos utilizar o software chamado *annovar*.  
Será necessário fazer download de alguns arquivos primeiro.

In [ ]:
# Download do annovar via wget do github
!wget -nv https://github.com/Varstation/T1-2020/raw/master/annovar/annovar.zip
!wget -nv http://www.openbioinformatics.org/annovar/download/NC_045512v2_avGene.txt.gz
!wget -nv http://www.openbioinformatics.org/annovar/download/NC_045512v2_avGeneMrna.fa.gz

In [ ]:
%%bash
#Descompartar
unzip annovar.zip
gunzip -c NC_045512v2_avGene.txt.gz > reference/NC_045512v2_avGene.txt
gunzip -c NC_045512v2_avGeneMrna.fa.gz > reference/NC_045512v2_avGeneMrna.fa
#Remover os arquivos zipados
rm annovar.zip *.gz

Comando necessário para acertar a referência de SARS-CoV-2.

In [ ]:

# Comando anotação
!sed -i 's/NC_045512.2/NC_045512v2/g' variants/{SAMPLE}_variants.vcf


Comando que faz a anotação.

In [ ]:

!annovar/table_annovar.pl -buildver NC_045512v2 -vcfinput variants/{SAMPLE}_variants.vcf reference/ -protocol avGene -operation g --polish

Veja que um arquivo com final *multianno.txt* foi criado na pasta *variants*.

In [ ]:
!tree

Ele está sujo, pois algumas posições tem a frequência do alelo igual a zero (AF=0). Basta filtrar as linhas com AF=1 para observar as variantes presentes no vírus SARS-CoV-2 presente nesta amostra.

In [ ]:
!head -n 1 variants/{SAMPLE}_variants.vcf.NC_045512v2_multianno.txt > variants/{SAMPLE}_final_variants_annotated.txt
!grep ';AF=1;' variants/{SAMPLE}_variants.vcf.NC_045512v2_multianno.txt >> variants/{SAMPLE}_final_variants_annotated.txt

Pronto, o arquivo *covid_patient(1 ou 2)_final_variants_annotated.txt* na pasta VARIANTS possui as variantes filtradas. Você pode baixá-lo no seu computador e abrir no excel para ver na forma de uma tabela.  
Perceba que a mudança no amino ácido da proteína está no final da coluna *AAChange.avgene* com o formato igual a este: E484K (significando que o aa E foi modificado por um aa K na posição 484.